# Pytorch 部分函数解释

## `nn.Conv2d` (二维卷积层)

函数声明：

```python
def nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True)
```

### 主要参数：

* `in_channels` (int): 输入图像的通道数 (例如，RGB 图像为 3)。
* `out_channels` (int): 卷积产生的输出通道数，即卷积核的数量。
* `kernel_size` (int or tuple): 卷积核的大小。可以是一个整数（例如 `3` 表示 3x3 卷积核）或一个元组 `(height, width)`。
* `stride` (int or tuple, optional): 卷积的步长。默认为 1。
* `padding` (int or tuple, optional): 输入图像边界的填充量。默认为 0。
* `dilation` (int or tuple, optional): 卷积核元素之间的间距。默认为 1。
* `groups` (int, optional): 从输入通道到输出通道的分组连接数。默认为 1。
* `bias` (bool, optional): 如果为 `True`，则向输出添加一个可学习的偏置。默认为 `True`。

下面我们给出一个使用案例，feel free to modify：


In [ ]:
import torch
import torch.nn as nn

# 假设输入是一个批量大小为 N，3 通道，高度为 64，宽度为 64 的图像
N, C_in, H_in, W_in = 4, 3, 64, 64
input_tensor = torch.randn(N, C_in, H_in, W_in)

# 定义一个 Conv2d 层
# 输入通道为 3，输出通道为 16，卷积核大小为 3x3，步长为 1，填充为 1
conv_layer = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)

# 将输入张量通过卷积层
output_tensor = conv_layer(input_tensor)

# 输出张量的形状
# O = (W - K + 2P) / S + 1
# H_out = (64 - 3 + 2*1) / 1 + 1 = 64
# W_out = (64 - 3 + 2*1) / 1 + 1 = 64
print(f"输入形状: {input_tensor.shape}") # torch.Size([4, 3, 64, 64])
print(f"输出形状: {output_tensor.shape}") # torch.Size([4, 16, 64, 64])


本质上来说，`nn.Conv2d` 在做如下的事情：

把输入为 `[N, C, H, W]` 的张量，经过卷积核的处理后，

输出为 `[N, output_channels, (H - kernel_size + 2 * padding) // stride + 1, (W - kernel_size + 2 * padding) // stride + 1]` 的张量。

## `nn.ConvTranspose2d` (二维转置卷积层)

`nn.ConvTranspose2d` 应用一个二维转置卷积操作。

函数声明：

```python
def nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride=1, padding=0, output_padding=0, groups=1, bias=True, dilation=1)
```

### 主要参数

* `in_channels` (int): 输入张量的通道数。
* `out_channels` (int): 转置卷积产生的输出通道数。
* `kernel_size` (int or tuple): 卷积核的大小。
* `stride` (int or tuple, optional): 卷积的步长。默认为 1。
* `padding` (int or tuple, optional): 输入的填充量。默认为 0。
* `output_padding` (int or tuple, optional): 添加到输出形状一侧的额外大小。默认为 0。这个参数用于在某些 `stride > 1` 的情况下精确控制输出尺寸。
* `groups` (int, optional): 从输入通道到输出通道的分组连接数。默认为 1。
* `bias` (bool, optional): 如果为 `True`，则向输出添加一个可学习的偏置。默认为 `True`。
* `dilation` (int or tuple, optional): 卷积核元素之间的间距。默认为 1。

下面我们给出一个使用案例，feel free to modify：


In [ ]:
import torch
import torch.nn as nn

# 假设输入是一个批量大小为 N，100 通道，高度为 1，宽度为 1 的潜向量 (通常用于 GAN 的生成器)
N, C_in, H_in, W_in = 4, 100, 1, 1
input_tensor = torch.randn(N, C_in, H_in, W_in)

# 定义一个 ConvTranspose2d 层
# 目标：将 1x1 的输入上采样到 4x4
# O = (I - 1) * S - 2P + K + OP
# 假设 K=4, S=1, P=0, OP=0 => O = (1-1)*1 - 0 + 4 + 0 = 4
trans_conv_layer1 = nn.ConvTranspose2d(in_channels=100, out_channels=512, kernel_size=4, stride=1, padding=0)
output1 = trans_conv_layer1(input_tensor)
print(f"输入形状 (1x1): {input_tensor.shape}")   # torch.Size([4, 100, 1, 1])
print(f"输出形状 (4x4): {output1.shape}")     # torch.Size([4, 512, 4, 4])

# 另一个例子：使用 stride=2 进行上采样，目标是将 4x4 上采样到 8x8
# K=4, S=2, P=1, OP=0 => O = (4-1)*2 - 2*1 + 4 + 0 = 3*2 - 2 + 4 = 6 - 2 + 4 = 8
input_tensor2 = torch.randn(N, 512, 4, 4)
trans_conv_layer2 = nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=4, stride=2, padding=1)
output2 = trans_conv_layer2(input_tensor2)
print(f"输入形状 (4x4): {input_tensor2.shape}")  # torch.Size([4, 512, 4, 4])
print(f"输出形状 (8x8): {output2.shape}")    # torch.Size([4, 256, 8, 8])

# 使用 output_padding 的情况
# 目标：将 5x5 上采样到 10x10，使用 K=3, S=2, P=1
# O_no_op = (5-1)*2 - 2*1 + 3 = 4*2 - 2 + 3 = 8 - 2 + 3 = 9
# 为了得到 10，我们需要 output_padding = 10 - 9 = 1
input_tensor3 = torch.randn(N, 128, 5, 5)
trans_conv_layer3 = nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=3, stride=2, padding=1, output_padding=1)
output3 = trans_conv_layer3(input_tensor3)
print(f"输入形状 (5x5): {input_tensor3.shape}")  # torch.Size([4, 128, 5, 5])
print(f"输出形状 (10x10): {output3.shape}") # torch.Size([4, 64, 10, 10])

本质上来说，`nn.ConvTranspose2d` 在做如下的事情：

把输入为 `[N, C, H, W]` 的张量，经过转置卷积核的处理后，

输出为 `[N, output_channels, (H - 1) * stride - 2 * padding + kernel_size + output_padding, (W - 1) * stride - 2 * padding + kernel_size + output_padding]` 的张量。

## `nn.BatchNorm2d` (二维批量归一化层)

`nn.BatchNorm2d` 对一个四维输入应用批量归一化。这个层通常被插入到卷积层和激活函数之间，以加速训练过程，提高模型的稳定性和性能。

它通过重新中心化和重新缩放输入来工作，使得每一层的输入的均值为0，方差为1。它还引入了两个可学习的参数 `gamma` (权重) 和 `beta` (偏置)，允许网络学习最佳的缩放和平移。

函数声明：

```python
def nn.BatchNorm2d(num_features, eps=1e-5, momentum=0.1, affine=True, track_running_stats=True)
```

### 主要参数

* `num_features` (int): 输入张量的通道数 (C from an input of shape `(N, C, H, W)`)。
* `eps` (float, optional): 为数值稳定性添加到分母的一个值。默认为 1e-5。
* `momentum` (float, optional): 用于计算 `running_mean` 和 `running_var` 的动量。默认为 0.1。
* `affine` (bool, optional): 一个布尔值，当设置为 `True` 时，此模块具有可学习的仿射参数 (`gamma` 和 `beta`)。默认为 `True`。
* `track_running_stats` (bool, optional): 一个布尔值，当设置为 `True` 时，此模块跟踪运行时的均值和方差。

下面我们给出一个使用案例，feel free to modify：



In [ ]:
import torch
import torch.nn as nn

# 假设输入是一个批量大小为 N，16 通道，高度为 32，宽度为 32 的特征图
N, C, H, W = 4, 16, 32, 32
feature_map = torch.randn(N, C, H, W)

# 定义一个 BatchNorm2d 层，对应 16 个特征通道
bn_layer = nn.BatchNorm2d(num_features=16)

# 将特征图通过批量归一化层
# 在训练模式下，它会计算当前批次的均值和方差，并更新运行时的均值和方差
# 同时使用当前批次的统计数据进行归一化
normalized_map_train = bn_layer(feature_map)

print(f"输入特征图形状: {feature_map.shape}")             # torch.Size([4, 16, 32, 32])
print(f"归一化后特征图形状 (训练): {normalized_map_train.shape}") # torch.Size([4, 16, 32, 32])

# 查看可学习参数 (如果 affine=True)
print(f"Gamma (weight): {bn_layer.weight.shape}") # torch.Size([16])
print(f"Beta (bias): {bn_layer.bias.shape}")     # torch.Size([16])

# 在评估模式下 (model.eval())
bn_layer.eval()
# 在评估模式下，它会使用训练过程中累积的运行时的均值和方差进行归一化
# 假设有一个新的评估数据
eval_feature_map = torch.randn(N, C, H, W)
normalized_map_eval = bn_layer(eval_feature_map)
print(f"归一化后特征图形状 (评估): {normalized_map_eval.shape}") # torch.Size([4, 16, 32, 32])

# 切换回训练模式
bn_layer.train()

在这个项目中，只需要知道：

`nn.BatchNorm2d` 会对输入的四维张量 `[N, C, H, W]` 进行批量归一化处理，输出的形状仍然是 `[N, C, H, W]`，但每个通道的均值为0，方差为1。